## Guidelines for Template
This template assists the CodeGen tool to do the round trip engineering of code management. Typically organizations with various roles like Data Analyst, Data Scientist, Operations Team , Support Desk etc. come out with their own standard and practices for automation, visualization and code Management. This sample template provides a guideline of how some of their guidelines can be converted to a template in assisting the CodeGen tool.

Here are the few basics:

Every code block requires a mandatory comment block (A) and a following code block (B).<br>
A.	Contents of the comment block:  A unique text based id which identifies the name of the code block in CodeGen.(Example:- UNIQUEID-ENCRYPTION, UNIQUEID-DERIVE COLUMN BY EXAMPLE).<br>
B.	Contents of the code block: Every cell needs an identifier in comment blocks. This identifier ensures that every cell is unique.

Typical rules in auto validating a template:
1.	1 to n number of comment blocks followed by one code block. This set of comment blocks will be shown to the user in CodeGen as a comment.
2.	UNIQUEIDs are unique across all the notebooks. If UNIQUEID repeats in different cells, the auto validation fails.
3.	All the comment blocks above a particular code block should have same unique id. If not text will not be displayed as a help text.
4.	ALLOWMERGEINTOANOTHERCELLFLAG-TRUE/FALSE lets the CodeGen to add the current code into existing cell. For example loading 18 tables in one block or creating 5 derived columns.
5.	CodeGen has the current context of the database name , schema name, table name , column name , stored procedure name and other data related variables. This allows context-sensitive variables to be injected into the code. The dynamic variables in the code can be identified with a tag start with ## and end with ##. Following are currently supported:
a.	##TABLENAME##
b.	##COLUMNNAME##
6.	In cell blocks allowed to be merged in one code block, if only a set of lines have to be repeated the block is identified by the section called ##REPEAT REGION STARTS## and ##REPEAT REGION ENDS##.
7.	The maximum suggested lines for code block is 200.
8.	Every code block should have unique functionality.

## Pip Initialization Code

In [2]:
!pip install azureml.dataprep
!pip install pyspark
!pip install azureml.core
import azureml.dataprep as dprep
import os
from pathlib import Path
import shutil
import tempfile
import sys

UNIQUEID-DATA INGESTION<br>
ALLOWMERGEINTOANOTHERCELLFLAG-TRUE

## Data Ingestion
Read SQL
Data Prep can also fetch data from SQL servers. Currently, only Microsoft SQL Server is supported.

To read data from a SQL server, first create a data source object that contains the connection information.

In [5]:
import azureml.dataprep as dprep

windows_domain = 'localhost'
windows_user = 'DataLakeManager\datalakeadmin'
windows_password = 'Infy1234567+'

secret = dprep.register_secret(value=windows_password, id="password")

ds = dprep.MSSQLDataSource(server_name=".",
                   database_name="AdventureWorksDW2017",
                   user_name=f'{windows_domain}\\{windows_user}',
                   password=secret)

ds.credentials_type = dprep.DatabaseAuthType.WINDOWS
##REPEAT REGION STARTS##
dflow##TABLENAME1## = dprep.read_sql(ds, "SELECT top 100 * FROM ##TABLENAME1##")
dflow##TABLENAME1##.head(5)
##REPEAT REGION ENDS##

UNIQUEID-ADD COLUMN USING EXPRESSION<br>
ALLOWMERGEINTOANOTHERCELLFLAG-FALSE
## Add Column using Expression
With Azure ML Data Prep you can add a new column to data with Dataflow.add_column by using a Data Prep expression to calculate the value from existing columns. This is similar to using Python to create a new script column except the Data Prep expressions are more limited and will execute faster. The expressions used are the same as for filtering rows and hence have the same functions and operators available.

In [7]:
#dflow = dprep.read_csv(path='/mnt/AdventureWorks/DimProduct.csv')
# USAGE EXAMPLE BELOW FOR A PARTICULAR TABLE/COLUMN
case_category = dflow##TABLENAME##.add_column(new_column_name='##COLUMNNAME## Category',
                                 prior_column='##COLUMNNAME##',
                                 expression=dflow##TABLENAME##['##COLUMNNAME##'].substring(0, 2))
case_category.head(5)

UNIQUEID-ASSERT<br>
ALLOWMERGEINTOANOTHERCELLFLAG-FALSE
## Assert
Azure ML Data Prep helps prevent broken pipelines and safeguard against bad data by supporting assertions.

In [9]:
# USAGE EXAMPLE BELOW FOR A PARTICULAR TABLE/COLUMN
# EXPRESSION CRITERIA in LINE #6 TO BE CUSTOM DEFINED BASED ON SCENARIO 
gb_asserted = dflow##TABLENAME##\
    .assert_value(
        columns='##COLUMNNAME##', 
        expression=dprep.f_or(dprep.value.is_null(), dprep.value <= 5),
        error_code='InvalidValue'
    )
gb_asserted.get_profile()
gb_errors = gb_asserted.filter(dprep.col('##COLUMNNAME##').is_error())
gb_errors.head(5)

UNIQUEID-CACHE<br>
ALLOWMERGEINTOANOTHERCELLFLAG-FALSE

## Cache
A Dataflow can be cached as a file on your disk during a local run by calling dflow_cached = dflow.cache(directory_path). Doing this will run all the steps in the Dataflow, dflow, and save the cached data to the specified directory_path. The returned Dataflow, dflow_cached, has a Caching Step added at the end. Any subsequent runs on on the Dataflow dflow_cached will reuse the cached data, and the steps before the Caching Step will not be run again.

Caching avoids running transforms multiple times, which can make local runs more efficient. Here are common places to use Caching:

after reading data from remote
after expensive transforms, such as Sort
after transforms that change the shape of data, such as Sampling, Filter and Summarize

In [11]:
# USAGE EXAMPLE BELOW FOR A PARTICULAR TABLE/COLUMN
# SAMPLE AND OTHER CRITERIA TO BE CUSTOM DEFINED BASED ON SCENARIO 

dflow##TABLENAME## = dflow##TABLENAME##.take_sample(probability=0.2, seed=7)
dflow##TABLENAME## = dflow##TABLENAME##.sort_asc(columns='##COLUMNNAME##')
dflow##TABLENAME## = dflow##TABLENAME##.keep_columns(['##COLUMNNAME##'])

cache_dir = str(Path(os.getcwd(), 'dataflow-cache'))
cache_dir

dflow##TABLENAME##_cached = dflow##TABLENAME##.cache(directory_path=cache_dir)

[s.step_type for s in dflow_cached._get_steps()]

os.listdir(cache_dir)

ddflow##TABLENAME##_cached.head(5)
dflow##TABLENAME##_cached_take = dflow##TABLENAME##_cached.take(10)
dflow##TABLENAME##_cached_skip = dflow##TABLENAME##_cached.skip(10).take(10)
dflow##TABLENAME##_cached_take = dflow##TABLENAME##_cached_take.to_pandas_dataframe()
dflow##TABLENAME##_cached_skip = dflow##TABLENAME##_cached_skip.to_pandas_dataframe()
dflow##TABLENAME##.head(5)
#shutil.rmtree(path=cache_dir)


UNIQUEID-COLUMN MANIPULATION<br>
ALLOWMERGEINTOANOTHERCELLFLAG-FALSE
## Column-manipulations(ColumnSelector)
ColumnSelector is a Data Prep class that allows us to select columns by name. The idea is to be able to describe columns generally instead of explicitly, using a search term or regex expression, with various options.

In [13]:
# USAGE EXAMPLE BELOW FOR A PARTICULAR TABLE/COLUMN
# EXPRESSION CRITERIA in LINE #5 TO BE CUSTOM DEFINED BASED ON SCENARIO 

from azureml.dataprep import ColumnSelector
column_selector = ColumnSelector(term="z",
                                 use_regex=False,
                                 ignore_case=True,
                                 match_whole_word=False,
                                 invert=False)
dflow##TABLENAME##_selected = dflow.keep_columns(column_selector)
dflow##TABLENAME##_selected.head(5)

UNIQUEID-COLUMNTYPE TRANSFORMS<br>
ALLOWMERGEINTOANOTHERCELLFLAG-FALSE 
## Column Type Transforms 
When consuming a data set, it is highly useful to know as much as possible about the data. Column types can help you understand more about each column, and enable type-specific transformations later. This provides much more insight than treating all data as strings.

In [15]:
# USAGE EXAMPLE BELOW FOR A PARTICULAR TABLE/COLUMN
# EXPRESSION CRITERIA in LINE #5 ONWARDS TO BE CUSTOM DEFINED BASED ON SCENARIO AND EXTRA LINES REMOVED

dflow##TABLENAME##_conversion_using_set = dflow##TABLENAME##.set_column_types({
    '##COLUMNNAME##': dprep.FieldType.INTEGER,
    '##COLUMNNAME##': dprep.FieldType.STRING,
    '##COLUMNNAME##': dprep.FieldType.BOOLEAN,
    '##COLUMNNAME##': (dprep.FieldType.INTEGER),
})
dflow##TABLENAME##_conversion_using_set.head(5)

UNIQUEID-CUSTOM PYTHON TRANSFORMS<br>
ALLOWMERGEINTOANOTHERCELLFLAG-FALSE
## Custom Python Transforms
There will be scenarios when the easiest thing for you to do is just to write some Python code. This SDK provides three extension points that you can use.

New Script Column<br/>
New Script Filter<br/>
Transform Partition

In [17]:
# USAGE EXAMPLE BELOW FOR A PARTICULAR TABLE/COLUMN
# EXPRESSION CRITERIA in LINE #5 ONWARDS TO BE CUSTOM DEFINED BASED ON SCENARIO AND EXTRA LINES REMOVED

col = dprep.col
pt_dflow = dflow##TABLENAME##

def transform(df, index):
    df['##COLUMNNAME##'].fillna('0',inplace=True)
    
    return df

dflow##TABLENAME## = pt_dflow.map_partition(fn=transform)
dflow##TABLENAME##.head(5)

UNIQUEID-DATA PROFILE<br>
ALLOWMERGEINTOANOTHERCELLFLAG-FALSE
## Data Profile
A DataProfile collects summary statistics on each column of the data produced by a Dataflow. This can be used to:

Understand the input data.
Determine which columns might need further preparation.
Verify that data preparation operations produced the desired result.
Dataflow.get_profile() executes the Dataflow, calculates profile information, and returns a newly constructed DataProfile.

In [19]:
dflow##TABLENAME##.get_profile()

UNIQUEID-READ FROM AND WRITE TO DATASTORES<br>
ALLOWMERGEINTOANOTHERCELLFLAG-FALSE
## Reading from and Writing to Datastores
A datastore is a reference that points to an Azure storage service like a blob container for example. It belongs to a workspace and a workspace can have many datastores.

A data path points to a path on the underlying Azure storage service the datastore references. For example, given a datastore named blob that points to an Azure blob container, a data path can point to /test/data/titanic.csv in the blob container.

In [21]:
# USAGE EXAMPLE BELOW FOR A PARTICULAR TABLE/COLUMN

subscription_id = '554376e1-5bbd-43f0-939f-d488aebb6f0f'
resource_group = 'CustomerExperience'
workspace_name = 'dataprep-centraleuap'

workspace = Workspace(subscription_id=subscription_id, resource_group=resource_group, workspace_name=workspace_name)
workspace.datastores

UNIQUEID-DERIVE COLUMN BY EXAMPLE<br>
ALLOWMERGEINTOANOTHERCELLFLAG-FALSE
## Derive column by example
One of the more advanced tools in Data Prep is the ability to derive columns by providing examples of desired results and letting Data Prep generate code to achieve the intended derivation.

In [23]:
# USAGE EXAMPLE BELOW FOR A PARTICULAR TABLE/COLUMN

df = dflow##TABLENAME##.head(10)
builder = dflow##TABLENAME##.builders.derive_column_by_example(source_columns=['##COLUMNNAME##'], new_column_name='##COLUMNNAME## New')
builder.add_example(source_data=df.iloc[1], example_value='Assets')
builder.preview() # will preview top 10 rows

UNIQUEID-EXTERNAL REFERENCES<br>
ALLOWMERGEINTOANOTHERCELLFLAG-FALSE
## External References
In addition to opening existing Dataflows in code and modifying them, it is also possible to create and persist Dataflows that reference another Dataflow that has been persisted to a .dprep file. In this case, executing this Dataflow will load and execute the referenced Dataflow dynamically, and then execute the steps in the referencing Dataflow.

In [25]:
# USAGE EXAMPLE BELOW FOR A PARTICULAR TABLE/COLUMN

dflow##TABLENAME## = dflow##TABLENAME##.drop_errors(['##COLUMNNAME##'], dprep.ColumnRelationship.ANY)
dflow##TABLENAME##_path = os.path.join(tempfile.gettempdir(), 'package.dprep')
dflow##TABLENAME##.save(dflow##TABLENAME##_path)
dflow##TABLENAME##_new = dprep.Dataflow.reference(dprep.ExternalReference(dflow_path))
dflow##TABLENAME##_new.head(5)
dflow##TABLENAME## = dflow##TABLENAME##.take(5)
dflow##TABLENAME##.save(dflow##TABLENAME##_path)

dflow##TABLENAME##_new.head(10)

UNIQUEID-FILTERING<br>
ALLOWMERGEINTOANOTHERCELLFLAG-FALSE
## Filtering
To filter rows, use DataFlow.filter. This method takes an Expression as an argument, and returns a new dataflow with the rows in which the expression evaluates to True. Expressions are built by indexing the Dataflow with a column name (dataflow['myColumn']) and regular operators (>, <, >=, <=, ==, !=).

In [27]:
# USAGE EXAMPLE BELOW FOR A PARTICULAR TABLE/COLUMN

dflow##TABLENAME##.filter(dprep.col('##COLUMNNAME##') != 'Assets').head(5)
dflow##TABLENAME## = dflow##TABLENAME##.filter((dflow##TABLENAME##['##COLUMNNAME##'] == '+') & (dflow##TABLENAME##['##COLUMNNAME##'] > 1))
dflow##TABLENAME##.head(5)

UNIQUEID-FUZZY GROUPING<br>
ALLOWMERGEINTOANOTHERCELLFLAG-FALSE
## Fuzzy Grouping
Unprepared data often represents the same entity with multiple values; examples include different spellings, varying capitalizations, and abbreviations. This is common when working with data gathered from multiple sources or through human input. One way to canonicalize and reconcile these variants is to use Data Prep's fuzzy_group_column (also known as "text clustering") functionality.

In [29]:
# USAGE EXAMPLE BELOW FOR A PARTICULAR TABLE/COLUMN

dflow = dprep.read_json(path='/FileStore/tables/json.json')
dflow.head(5)
dflow##TABLENAME##_clean = dflow##TABLENAME##.fuzzy_group_column(source_column='inspections.business.city',
                                       new_column_name='city_grouped',
                                       similarity_threshold=0.8,
                                       similarity_score_column_name='similarity_score')
dflow_clean.head(5)

UNIQUEID-IMPUTE MISSING VALUES<br>
ALLOWMERGEINTOANOTHERCELLFLAG-FALSE
## Impute missing values
Azure ML Data Prep has the ability to impute missing values in specified columns. In this case, we will attempt to impute the missing Latitude and Longitude values in the input data.

In [31]:

dflow##TABLENAME## = dflow##TABLENAME##.keep_columns(['##COLUMNNAME##', '##COLUMNNAME##', '##COLUMNNAME##'])
dflow##TABLENAME## = dflow##TABLENAME##.to_number(['FinishedGoodsFlag'])
dflow##TABLENAME##.head(5)
dflow##TABLENAME##_mean = dflow##TABLENAME##.summarize(group_by_columns=['##COLUMNNAME##'],
                       summary_columns=[dprep.SummaryColumnsValue(column_id='##COLUMNNAME##',
                                                                 summary_column_name='##COLUMNNAME##_MEAN',
                                                                 summary_function=dprep.SummaryFunction.MEAN)])
dflow##TABLENAME##_mean = dflow##TABLENAME##_mean.filter(dprep.col('##COLUMNNAME##') == 'FALSE')
ddflow##TABLENAME##_mean.head(1)

UNIQUEID-INSPECTOR<br>
ALLOWMERGEINTOANOTHERCELLFLAG-FALSE
## Inspector
An Inspector collects specific statistics on specified columns of a Dataflow. This can be used to understand the input data under contraints.

Dataflow.execute_inspector() executes the Inspector. It takes an Inspector as an argument.

In [33]:

bw_inspector = dprep.BoxAndWhiskerInspector("##COLUMNNAME##")
bw_result = dflow##TABLENAME##.execute_inspector(bw_inspector)

bw_result

column_stats_inspector = dprep.ColumnStatsInspector("##COLUMNNAME##")
column_stats_result = dflow##TABLENAME##.execute_inspector(column_stats_inspector)

batch_result = dflow##TABLENAME##.execute_inspectors([bw_inspector, column_stats_inspector])
batch_result

batch_result[column_stats_inspector]

UNIQUEID-SUMMARIZE<br>
ALLOWMERGEINTOANOTHERCELLFLAG-FALSE
## Summarize
Azure ML Data Prep can help summarize your data by providing you a synopsis based on aggregates over specific columns.

In [35]:

dflow##TABLENAME##_summarize = dflow##TABLENAME##.summarize(
        summary_columns=[
            dprep.SummaryColumnsValue(
                column_id='##COLUMNNAME##',
                summary_column_name='##COLUMNNAME## Type ID Counts', 
                summary_function=dprep.SummaryFunction.COUNT)],
        group_by_columns=['##COLUMNNAME##'])
dflow##TABLENAME##_summarize.head(10)

UNIQUEID-REPLACE,FILL,ERROR<br>
ALLOWMERGEINTOANOTHERCELLFLAG-FALSE
## Replace, Fill, Error
replace - use this method to replace a value with another value. You can also use this to replace null with a value, or a value with null<br/>
error - use this method to replace a value with an error.<br/>
fill_nulls - this method lets you fill all nulls in a column with a certain value.<br/>
fill_errors - this method lets you fill all errors in a column with a certain value.

In [37]:

dflow##TABLENAME## = dflow##TABLENAME##.replace('##COLUMNNAME##', 'Assets', 'Current Assets')
head = dflow##TABLENAME##.head(5)
dflow##TABLENAME## = dflow##TABLENAME##.error('##COLUMNNAME##', 210, 'Invalid value')
head = dflow##TABLENAME##.head(5)
dflow##TABLENAME## = dflow##TABLENAME##.fill_nulls('##COLUMNNAME##', 'N/A')
head = dflow##TABLENAME##.head(5)

dflow##TABLENAME## = dflow##TABLENAME##.fill_errors('##COLUMNNAME##', -1)
head = dflow##TABLENAME##.head(5)
head

UNIQUEID-JOINS<br>
ALLOWMERGEINTOANOTHERCELLFLAG-FALSE
## Joins
In Data Prep you can easily join two Dataflows.

In [39]:

dflow##TABLENAME1##.join(
    left_dataflow=dflow##TABLENAME1##,
    right_dataflow=dflow##TABLENAME2##,
    join_key_pairs=[
        ('##COLUMNNAME1##', '##COLUMNNAME2##')
    ]
).head(5)

UNIQUEID-LABEL ENCODER<br>
ALLOWMERGEINTOANOTHERCELLFLAG-FALSE
## Label Encoder
Data Prep has the ability to encode labels with values between 0 and (number of classes - 1) using label_encode.

In [41]:
from datetime import datetime

dflow##TABLENAME## = dflow##TABLENAME##.label_encode(source_column='##COLUMNNAME##', new_column_name='##COLUMNNAME## Label')
dflow##TABLENAME##.head(2)
dflow##TABLENAME##.to_json()
builder = dflow##TABLENAME##.builders.label_encode(source_column='##COLUMNNAME##', new_column_name='##COLUMNNAME## Label')
builder.learn()
builder.encoded_labels
encoded_labels = builder.encoded_labels
encoded_labels['True'] = 1
encoded_labels['False'] = 0
builder.encoded_labels = encoded_labels
builder.encoded_labels

UNIQUEID-MIN-MAX SCALER<br>
ALLOWMERGEINTOANOTHERCELLFLAG-FALSE 
## Min-Max Scaler 
The min-max scaler scales all values in a column to a desired range (typically [0, 1]). This is also known as feature scaling or unity-based normalization. Min-max scaling is commonly used to normalize numeric columns in a data set for machine learning algorithms.

In [43]:
dflow##TABLENAME## = dflow##TABLENAME##.keep_columns(columns=['##COLUMNNAME##', '##COLUMNNAME##', '##COLUMNNAME##'])
dflow##TABLENAME## = dflow##TABLENAME##.to_number(columns=['##COLUMNNAME##', '##COLUMNNAME##'])
dflow##TABLENAME## = dflow##TABLENAME##.min_max_scale(column='##COLUMNNAME##')
dflow##TABLENAME##.head(5)

UNIQUEID-MOUNTING DATAFLOWS<br>
ALLOWMERGEINTOANOTHERCELLFLAG-FALSE
## Mounting dataflows
create a dataflow that points to some images in an Azure Blob Container. The dataflow will contain 1 column called Path with StreamInfo objects inside it, which references the source data.
Note: mounting only works on Unix or Unix-like operating systems.

In [45]:
dflow##TABLENAME## = dprep.Dataflow.get_files('https://smdwstorage.blob.core.windows.net/validationdb/*.jpg')
dflow##TABLENAME##.head(5)
from azureml.dataprep.api.functions import get_portable_path
from azureml.dataprep.api.expressions import col
portable_path_col = 'Portable Path'
dflow = dflow.add_column(
    expression=get_portable_path(col('Path')), 
    new_column_name=portable_path_col, 
    prior_column='Path')
dflow.head(5)


UNIQUEID-ONE HOT ENCODER<br>
ALLOWMERGEINTOANOTHERCELLFLAG-FALSE
## One Hot Encoder
Azure ML Data Prep has the ability to perform one hot encoding on a selected column using one_hot_encode. The result Dataflow will have a new binary column for each categorical label encountered in the selected column.

In [47]:
dflow##TABLENAME##_result = dflow##TABLENAME##.one_hot_encode(source_column='##COLUMNNAME##')
dflow##TABLENAME##_result.head(5)

UNIQUEID-QUANTILE TRANSFORMATION<br>
ALLOWMERGEINTOANOTHERCELLFLAG-FALSE
## Quantile Transformation
DataPrep has the ability to perform quantile transformation to a numeric column. This transformation can transform the data into a normal or uniform distribution. Values bigger than the learnt boundaries will simply be clipped to the learnt boundaries when applying quantile transformation.

In [49]:

dflow##TABLENAME## = dflow##TABLENAME##.quantile_transform(source_column='##COLUMNNAME##', new_column='##COLUMNNAME##_uniform', quantiles_count=5)
dflow##TABLENAME## = dflow##TABLENAME##.quantile_transform(source_column='##COLUMNNAME##', new_column='##COLUMNNAME##_normal', 
                           quantiles_count=5, output_distribution="Normal")
dflow##TABLENAME##.get_profile()

UNIQUEID-RANDOM SPLIT<br>
ALLOWMERGEINTOANOTHERCELLFLAG-FALSE
## Random Split
Azure ML Data Prep provides the functionality of splitting a data set into two. When training a machine learning model, it is often desirable to train the model on a subset of data, then validate the model on a different subset.

In [51]:
dflow##TABLENAME## = dflow##TABLENAME##.keep_columns(['##COLUMNNAME##', '##COLUMNNAME##'])
profile = dflow##TABLENAME##.get_profile()
print('Row count: %d' % (profile.columns['##COLUMNNAME##'].count))

(dflow##TABLENAME##_test, dflow##TABLENAME##_train) = dflow##TABLENAME##.random_split(percentage=0.1)
profile_test = dflow##TABLENAME##.get_profile()
print('Row count of "test": %d' % (profile_test.columns['##COLUMNNAME##'].count))


UNIQUEID-REPLACE DATASOURCE REFERENCE<br>
ALLOWMERGEINTOANOTHERCELLFLAG-FALSE
## Replace DataSource Reference
A common practice when performing DataPrep is to build up a script or set of cleaning operations on a smaller example file locally. This is quicker and easier than dealing with large amounts of data initially.

In [53]:

dflow##TABLENAME## = dflow##TABLENAME##.to_pandas_dataframe()
dflow##TABLENAME##_dropped = dflow##TABLENAME##.drop_columns(['##COLUMNNAME##', '##COLUMNNAME##', '##COLUMNNAME##', '##COLUMNNAME##', '##COLUMNNAME##','##COLUMNNAME##'])
sctb = dflow##TABLENAME##_dropped.builders.set_column_types()
sctb.learn(inference_arguments=dprep.InferenceArguments(day_first=False))
dflow##TABLENAME##_typed = sctb.to_dataflow()
dflow##TABLENAME##_typed.head(5)

UNIQUEID-SEMANTIC TYPES<br>
ALLOWMERGEINTOANOTHERCELLFLAG-FALSE
## Semantic Types
Some string values can be recognized as semantic types. For example, email addresses, US zip codes or IP addresses have specific formats that can be recognized, and then split in specific ways.

In [55]:
dflow = dprep.read_json(path='/FileStore/tables/json.json')

profile = dflow##TABLENAME##.get_profile(include_stype_counts=True)

print("row count: " + str(profile.row_count))
profile.stype_counts
[t.name for t in dprep.SType]
stypes_counts = profile.stype_counts
all_match = [
    (column, stypes_counts[column][0].stype)
    for column in stypes_counts
    if profile.row_count - profile.columns[column].empty_count == stypes_counts[column][0].count
]
all_match

UNIQUEID-WORKING WITH FILE STREAMS<br>
ALLOWMERGEINTOANOTHERCELLFLAG-FALSE
## Working With File Streams
Location is the type of storage where the stream is located (e.g. Azure Blob, Local, or ADLS); ResouceIdentifier is the name of the file within that storage, such as a file path; and Arguments is a list of arguments required to load and read the file.

On their own, StreamInfo objects are not particularly useful; however, you can use them as input to other functions.

In [57]:
dflow##TABLENAME##= dprep.Dataflow.get_files(path='https://smdwstorage.blob.core.windows.net/validationdb/*.csv')
dflow##TABLENAME## = dflow##TABLENAME##.add_column(expression=dprep.get_stream_name(dflow##TABLENAME##['Path']),
                         new_column_name='FilePath',
                         prior_column='Path')
first_file_path = dflow##TABLENAME##.head(1)['FilePath'][0]
first_file_name = os.path.basename(first_file_path)
dflow##TABLENAME## = dflow##TABLENAME##.derive_column_by_example(new_column_name='FileName',
                                       source_columns=['FilePath'],
                                       example_data=(first_file_path, first_file_name))
dflow##TABLENAME## = dflow##TABLENAME##.drop_columns(['FilePath'])
dflow##TABLENAME##_write = dflow##TABLENAME##.write_to_csv(directory_path=dprep.LocalFileOutput('./test_out/'))
dflow##TABLENAME##_written_files = dprep.read_csv('./test_out/part-*')
dflow##TABLENAME##_written_files.head(5)
